# Table of Contents
[1. Model Preparation](#1.-Model-Preperation)
<br>
* [1.1 Reviewing, Splitting data set](#1.1-Reviewing,-splitting-dataset-into-7:3-for-training-and-testing.)
* [1.2 Plotting features against target feature](#1.2-Plot-to-compare-all-features-to-target-feature-to-help-make-decisions-to-keep-for-the-models.)
    * [1.2.1 Plotting datetime feature against target feature](#Plotting-datetime-feature-against-target-feature)
    * [1.2.2 Plotting numerical features against target feature](#Plotting-numerical-features-against-target-feature)
    * [1.2.3 Plotting categorical features against target feature](#Plotting-categorical-features-against-target-feature)
* [1.3. Summary of all features](#1.3.-Summary-of-all-features)
    * [1.3.1 Numerical Features](#Numerical-Features)
    * [1.3.1 Cateogrical Features](#Categorical-Features)
* [2. Linear Regression](#2.-Linear-Regression)
* [3. Route model and taking the proportion of the prediction to calculate a journey time for the user](#3.-Route-model-and-taking-the-proportion-of-the-prediction-to-calculate-a-journey-time-for-the-user.)
    * [3.1 Calculating the proportion of each stop from the overall trip](#3.1-Calculating-the-proportion-of-each-stop-from-the-overall-trip.)
* [4. Random Forest & Decision Trees](#4.-Random-Forest-&-Decision-Trees)
* [5. Stop pair model](#5.-Stop-pair-model)
    * [5.1 First version of paired stop approach](#5.1-First-version-of-paired-stop-approach)
    * [5.2.1 Setting up for 46a stop pair models using first approach](#5.2.1-Setting-up-for-46a-stop-pair-models-using-first-approach)
    * [5.3 Stop pair based on entire leavetimes](#5.3-Stop-pair-based-on-entire-leavetimes)

Establishing a connection with sqlite database

In [ ]:
# import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sqlite3
import pickle

# from sagemaker import get_execution_role
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from math import log
from statistics import stdev
from statistics import mode


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Connecting to s3
# role = get_execution_role()
# bucket='sagemaker-studio-520298385440-7in8n1t299'
# data_key = 'route_46a.feather'
# data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except 'Error' as e:
        print(e)
        
    return conn

In [ ]:
# create connection to db
db_file = "C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/dublinbus.db"
conn = create_connection(db_file)

In [ ]:
# initialise query
query = """
SELECT leavetimes.*, weather.*
FROM leavetimes, weather
WHERE TRIPID in  
    (SELECT TRIPID
    FROM trips
    WHERE LINEID = '46A' AND DIRECTION = '1')
AND leavetimes.DAYOFSERVICE = weather.dt;
"""

In [ ]:
# execute query and read into dataframe
query_df = pd.read_sql(query, conn)

# 1. Model Preperation

In [ ]:
# Loading file
df = query_df

## 1.1 Reviewing, splitting dataset into 7:3 for training and testing.

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
# Missing values
df.isnull().sum()

In [ ]:
# Unique types for each feature
df.nunique()

In [ ]:
# Datatypes and convert
df.dtypes

In [ ]:
# Rows and columns
df.shape

**Review so far:**
<br>
There are no more missing values and the constant columns have been removed.
* Remove index, index, dt.
* Investigate level_0.
* Convert the following to categorical: DAYOFWEEK, MONTHOFSERVICE, PROGRNUMBER, STOPPOINTID, VEHICLEID, IS_HOLIDAY, IS_WEEKDAY, TRIPID, weather_id, weather_main, weather_description
* We have data for most of the days of the year and for each month.


In [ ]:
df = df.drop(['index', 'level_0', 'dt'], axis=1)

In [ ]:
# Sorting by trip then dayofservice
df = df.sort_values(by=['TRIPID', 'DAYOFSERVICE', 'PROGRNUMBER'])

In [ ]:
# Creating features
categorical_features = ['DAYOFWEEK', 'MONTHOFSERVICE', 'PROGRNUMBER', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID',
                       'IS_HOLIDAY', 'IS_WEEKDAY', 'TRIPID', 'VEHICLEID', 'weather_id', 'weather_main', 'weather_description']

datetime_features = ['DAYOFSERVICE']

numerical_features = ['PLANNEDTIME_ARR', 'ACTUALTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_DEP',
                     'DWELLTIME', 'PLANNEDTIME_TRAVEL', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'clouds_all']

target_feat = 'ACTUALTIME_TRAVEL'

In [ ]:
# Converting object to categorical
for column in categorical_features:
    df[column] = df[column].astype('category')
    
# Converting dayofservice to datetime
df['DAYOFSERVICE'] = pd.to_datetime(df['DAYOFSERVICE'])

In [ ]:
# Replacing PROGRNUMBER equal to 1 of ACTUALTIME_TRAVEL with 0
df.loc[df['PROGRNUMBER'] == '1', 'ACTUALTIME_TRAVEL'] = 0
df.loc[df['PROGRNUMBER'] == '1', 'PLANNEDTIME_TRAVEL'] = 0

In [ ]:
df.loc[df['PLANNEDTIME_TRAVEL'] < 0, 'PLANNEDTIME_TRAVEL'] = 0
df.loc[df['ACTUALTIME_TRAVEL'] < 0, 'ACTUALTIME_TRAVEL'] = 0

In [ ]:
# Making new feature for previous stoppointid and let those with PROGRNUMBER = 1 to 0
# df['PREVIOUS_STOPPOINTID'] = df['STOPPOINTID'].shift()
# first_stop = {'0':'0'}
# df['PREVIOUS_STOPPOINTID'] = df['PREVIOUS_STOPPOINTID'].cat.add_categories(first_stop)
# df.loc[df['PROGRNUMBER'] == '1', 'PREVIOUS_STOPPOINTID'] = '0'

<br><br>
Setting the target feature as _y and x_ as the remaining features in the dataframe. 
<br><br>

In [ ]:
df.set_index(np.random.permutation(df.index))
# sort the resulting random index
df.sort_index(inplace=True)
df.head(5)

In [ ]:
# Creating y and x axis
target_feature = df['ACTUALTIME_TRAVEL']
y = pd.DataFrame(target_feature)
X = df.drop(['ACTUALTIME_TRAVEL'], axis=1)

# Splitting dataset for train and testing data by 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Printing shape of the new split data
print("The original range is: ",df.shape[0])
print("The training range (70%):\t rows 0 to", round(X_train.shape[0]))
print("The test range (30%): \t rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

In [ ]:
X_train.head(5)

## 1.2 Plot to compare all features to target feature to help make decisions to keep for the models.

#### Plotting datetime feature against target feature

In [ ]:
# Plot datetime feature against target feature
X_train.DAYOFSERVICE = pd.to_numeric(X_train.DAYOFSERVICE)
df_temp = pd.concat([X_train['DAYOFSERVICE'], y_train], axis=1)
correlation_dt = df_temp[['DAYOFSERVICE', 'ACTUALTIME_TRAVEL']].corr(method='pearson')
correlation_dt

In [ ]:
fig = plt.figure()
ax = fig.add_subplot
df_temp.plot(kind='scatter', x='DAYOFSERVICE', y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', 'DAYOFSERVICE']].corr().to_numpy()[0,1], figsize=(15, 8)) 

#### Plotting numerical features against target feature

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    correlation_dt = df_temp[[column, 'ACTUALTIME_TRAVEL']].corr(method='pearson')
    print('\n',correlation_dt)

Using pearson correlation, we see that the correlation between 

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    correlation_dt = df_temp[[column, 'ACTUALTIME_TRAVEL']].corr(method='spearman')
    print('\n',correlation_dt)

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='pearson').to_numpy()[0,1], figsize=(12, 8)) 

In [ ]:
for column in numerical_features:
    df_temp = pd.concat([X_train[column], y_train], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='spearman').to_numpy()[0,1], figsize=(12, 8)) 

In [ ]:
df_numeric = df[numerical_features]
for feature in df_numeric:
    df_numeric[feature] = np.log(df_numeric[feature])
df_numeric['ACTUALTIME_TRAVEL'] = np.log(df['ACTUALTIME_TRAVEL'])

In [ ]:
# Creating y and x axis
target_feature_numeric = df_numeric['ACTUALTIME_TRAVEL']
y_numeric = pd.DataFrame(target_feature_numeric)
X_numeric = df_numeric.drop(['ACTUALTIME_TRAVEL'], axis=1)

# Splitting dataset for train and testing data by 70/30
X_train_numeric, X_test_numeric, y_train_numeric, y_test_numeric = train_test_split(X_numeric, y_numeric, test_size=0.3, random_state=1)

# Printing shape of the new split data
print("The original range is: ",df.shape[0])
print("The training range (70%):\t rows 0 to", round(X_train_numeric.shape[0]))
print("The test range (30%): \t rows", round(X_train_numeric.shape[0]), "to", round(X_train_numeric.shape[0]) + X_test_numeric.shape[0])

for column in numerical_features:
    df_temp = pd.concat([X_train_numeric[column], y_train_numeric], axis=1)
    fig = plt.figure()
    ax = fig.add_subplot
    df_temp.plot(kind='scatter', x=column, y='ACTUALTIME_TRAVEL', label = "%.3f" % df_temp[['ACTUALTIME_TRAVEL', column]].corr(method='spearman').to_numpy()[0,1], figsize=(12, 8)) 

#### Plotting categorical features against target feature

In [ ]:
year_features = ['DAYOFWEEK', 'IS_HOLIDAY', 'IS_WEEKDAY', 'MONTHOFSERVICE', 'weather_id', 'weather_main', 'weather_description']

for feature in year_features:
    print(feature)
    df_temp = pd.concat([X_train, y_train], axis=1)
    unique = df_temp[feature].unique()
    list_average = []
    
    for value in unique:
        list_values = df_temp[df_temp[feature]== value]['ACTUALTIME_TRAVEL'].tolist()
        length_list = len(list_values)
        average =  sum(list_values)/length_list
        list_average += [average]
#         print(f'Sum of values / list of values: \n {sum(list_values)} / {length_list}')
#         print(f'Average ACTUALTIME_TRAVEL: {average}, \n')
        
    # taken from https://pythonspot.com/matplotlib-bar-chart/
    y_pos = np.arange(len(unique))
    plt.bar(y_pos, list_average, align='center')
    plt.xticks(y_pos, unique)
    plt.ylabel('Usage')
    plt.title(feature)
    plt.xticks(rotation=90)

    plt.show()

I think there is some outliers in ACTUALTME_TRAVEL. The averages are all in negatives which suggests that the travel times. Would that be an outlier if the negative values are very great? 
<br><br>
**DAYOFWEEK:**
The lowest average is Sunday and the busiest is Monday. So it does make a difference.
<br><br>
**IS_WEEKDAY:**
The same comment that there is a difference in average times.
<br><br>
**MONTHOFSERVICE:**
Interestingly enough, they have difference averages depending on each month with August being the least busiest and April being the busiest. It must have something to do with the weather maybe?


In [ ]:
# Average time for each vehicle id
df_temp = pd.concat([X_train, y_train], axis=1)
vehicleid = df_temp['VEHICLEID'].unique().tolist()
for id_ in vehicleid:
    print(f'VEHICLEID: {id_}')
    list_values = df_temp[df_temp['VEHICLEID']== id_]['ACTUALTIME_TRAVEL'].tolist()
    length_list = len(list_values)
    average =  sum(list_values)/length_list
    print(f'Average ACTUALTIME_TRAVEL: {average} \n')

In [ ]:
# Making dummy variables for categorical 
cat = ['DAYOFWEEK', 'MONTHOFSERVICE', 'PROGRNUMBER', 'STOPPOINTID', 'IS_HOLIDAY', 'IS_WEEKDAY', 'weather_id', 'weather_main', 'weather_description']
df_temp = pd.concat([X_train, y_train], axis=1)
df_copy = df_temp.copy()
df_copy = df_copy[cat]
df_copy = pd.get_dummies(df_copy)
df_copy = pd.concat([df_copy, y_train], axis=1)

categorical_corr = df_copy.corr()['ACTUALTIME_TRAVEL'][:]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(categorical_corr)

In [ ]:
categorical_list = categorical_corr[categorical_corr > 0.04].index.tolist()
categorical_list.remove('ACTUALTIME_TRAVEL')

In [ ]:
categorical_list

## 1.3. Summary of all features
<br><br>
#### Numerical Features
<br><br>

**DayOfService:**
* The correlation to the target feature is very low of 0.03806.
* Don't see it being a useful feature for the target feature. 
* Plot represents a straight line, which suggests little to no correlation.
* Conclusion: dropped because of the low correlation score. 

**PlannedTime_Arr:**
* There is very low correlation against the target feature though it gets better using spearman correlation.
* After logging the data, the correlation plot did not make a huge difference when using the spearman method to plot it for the second time. 
* Pearson and spearman plot pre log suggests little correlation as it is a continuous straight line. However, this shouldn't mean it should be dropped.
* When most values in the target feature fell less than 10, we see that the plannedtime arrival values increasing, it didn't change much. This would be due to the fact that the target feature is the difference between times so it would make sense that the relationship is poor.
* After logging the data, the plot is more spread out instead of a straight line, but the correlation score still shows a similar low score with a .02 difference using the spearman method. 
* Conclusion: However, this will be dropped.

**ActualTime_Arr:**
* Compared to Planned time arrival feature, the pearson correlation score is poorer but the spearman scores are more similar pre log. 
* It is similar to planned time arrival in that the plot represents a straight line, that suggests a poor relationship with the target feature. 
* After logging the data, it is found that the plot is more spread out. The score using spearman is not much different pre logging the data. 
* However, it would be unwise to drop this feature as it I feel it would serve good purpose for the target feature for predicting the prediction time for the next stop. 
* Conclusion: this will be dropped.

**PlannedTime_Dep:**
* Planned time departure has little correlation with the target feature after looking at spearman and pearsons. 
* It doesn't have a linear relationship and the straight line on the plot of both methods proves this.
* However, when plotted using the logged values we see that the correlation score hasn't changed but the data is more spread out. 
* This doesn't change the relationship much, however. 
* Even so, this will be kept as I feel it would help the predictions. Having the planned time departures would help skew a better result because it would relatively be close to the actual time departure even though it is just an estimate.
* Conclusion: this will be dropped 

**ActualTime_Dep:**
* Actual time departure is again, more or less the same. It represents the departure for these times at a particular stop to go to the next stop. It is strange that the correlation is so low even after logging the data but it would make sense as you wouldn't expect there to be a linear relationship.
* The plot is similar to the rest of the previous features mentioned so far. 
* However, it will still be kept because I feel it would still be a useful feature for predicting a time in seconds. 
* By taking the actual time departure for a particular stop it may help.
* Conclusion: this will be dropped.

**Dwell Time:**
* Dwell time has a 0.03 coorelation score with the target feature. It suggests on the graph that the time for dwell time equal to 0 then the more the target feature time increases. It might suggest traffic times where if a bus is full then it might be due to rush hour? busy hours?
* Plotting against the target feature after logging the data gives similar scores using the spearman correlation method. However we see the graph differing from pre log plot. It is more grouped up together compared to the previous graph plot.
* Because the score is more fairer compared to the previous, it will be useful to keep it for the modelling.
* Conclusion: dropped.

**PlannedTime_Travel:**
* When plotting using the pearse correlation method, it gave a correlation of 0.2. This time it is the highest correlation and we see a small linear relationship.
* The time for planned time travel, as it increases, so does the target feature. It gives us an indication of that slight linear relationship.
* Using spearmans to graph the correlation gave us a 0.7 score which is a good indication that the two features has a linear relationship.
* Because of this, this feature will be dropped.

**Temp:**
* Temp  has a negative 0.009 correlation with the target feature and an even poorer linear relationship at -.002.
* This indicates a poor linear/monotonic relationship and it will not serve useful for the model.
* The graph plots does not give anymore useful information that would give further evidence that it should be kept.
* Conclusion: drop.

**Pressure:**
* It also has a negative linear relationship with the target feature.
* When looking at the graph plots for both spearman and pearsons, it does not give any further insights.
* For this reason, this feature will be dropped.

**Humidity:**
* Humidity does not have a strong relationship with the target feature, be it linear or monotonic.
* The reason being the correlation using both methods fell < 0.00. 
* Unfortunately, the graph does not represent anything useful either.
* When looking at the logged data plots however, there is a slight difference however it is not signficant enough that this feature should still be kept as there is no distinct relationship that we can see.
* Conclusion: drop.

**Windspeed:**
* No linear relationship.
* Indicates a small monotonic relationship.
* This means that as the windspeed value increases, the value of the target feature tends to be higher as well.
* But a spearman correlation of 0.01 is not strong enough of a feature to keep.
* Conclusion: drop

**Wind_Deg:**
* This feature will be dropped immediately as the correalations are both <0.000.

**Rain_1H:**
* It doesn't have a strong linear relationship but it shows spearmans correlation some promising results when the data has been logged.


<br><br>
#### Categorical Features
<br><br>
**DayOfWeek:**
* In the graph we see the actual time travel increasing during weekdays and slowly the travel time is less during weekends. 
* This suggests a relationship between the days of the week and the target feature in which weekdays have a higher tendency for the actualtime travel feature to be higher.
* Conclusion: this will be kept.

**MonthofService:**
* In the graph, we don't really see a connection between each month against the target feature even if it is in order. 
* The overall actual travel time is higher in february before it dips, then rising during winter season.
* The correlation score seems to be poor also for each month. 
* This feature will still be kept. 

**Progrnumber:**
* Most progrnumbers will be dropped as a lot of the correlations are <0.00.
* For this reason, this feature will be dropped.
    
**StoppointID:**
* Similarly to progrnumbers, there are a lot of low correlations falling <0.00.
* Most stoppoint numbers are <0.00 correlation.
* This indicates a very low relationship with the target feature. 
* For this reason, this feature will be dropped, except for those with a correlation > 0.04
    
**Is_Holiday:**
* After analyzing the graph, we see a relationship between the target feature and whether or not the time falls under a holiday date (non-school holiday).
* If it a non holiday, the actual time travel increases. 
* If it is a holiday, the actual time travel decreases. 
* This means that less people are using public transport if it is a holiday date.
* For this reason, this feature will be kept.

**Is_Weekday:**
* Like Is_Holiday, we see a relationship between the target feature and whether or not the time is during a weekday or not. 
* We see a contrast between the two values in which 1, being a weekday, has a higher actual time travel, vice versa.
* For this reason, it is a good indication of a relationship to the target feature.
* Therefore, this feature will be kept. 

**VehicleID:**
* When looking at the different averages, we see that the average differences are not big.
* For this reason, it may be best to drop this feature because it doesn't give any indication it would be a useful feature to help the prediction models.


## 1.4 Cleaning up features

### Setting low correlation features - keep

In [ ]:
# Categorical features
low_corr_categorical = ['DAYOFWEEK', 'MONTHOFSERVICE', 'IS_HOLIDAY', 'IS_WEEKDAY'] 

### Setting low correlation features - drop

In [ ]:
# Numerical features
low_corr_numerical = ['PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP','PLANNEDTIME_TRAVEL']

low_corr = ['DAYOFSERVICE', 'VEHICLEID', 'TRIPID', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID', 'PROGRNUMBER', 'temp', 'pressure', 'humidity', 
            'wind_speed', 'wind_deg', 'weather_id', 'weather_description', 'clouds_all', 'PREVIOUS_STOPPOINTID', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP',
           'PLANNEDTIME_TRAVEL', 'DWELLTIME']

### Setting high correlation  features

In [ ]:
# Numerical features 
high_corr_numerical = ['DWELLTIME', 'PLANNEDTIME_TRAVEL']

### Dropping features & setting dummy features

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(low_corr, 1)
df_copy

In [ ]:
df_copy = pd.get_dummies(df_copy)
df_copy

### Training & Testing data

In [ ]:
# All features
features = df_copy.columns.tolist()
features

In [ ]:
datas = {'ACTUALTIME_TRAVEL': df_copy['ACTUALTIME_TRAVEL']}
y = pd.DataFrame(data=datas)
X = df_copy.drop(['ACTUALTIME_TRAVEL'],1)

In [ ]:
# Splitting the dataset into 2 datasets: 
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

print("The Original range of the dataset: ",df.shape[0])
print("The Training range taken from dataset: (70%): rows 0 to", round(X_train.shape[0]))
print("The Test range taken from dataset: (30%): rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

In [ ]:
print("\nDescriptive features in X:\n", X_train.head(5))
print("\nTarget feature in y:\n", y_train.head(5))

In [ ]:
# I will reset the indexes of the training and test splits so we can see the X_train printout
# We will see that they are no longer in order and the next markdown cell I will reset the indexes.
X_train.head(5)

In [ ]:
# Using .reset_index 
# We see that they are in order again. 
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
X_train.head(10)

***

<br><br>
# 2. Linear Regression

In this section, I will be preparating a linear regression model. I will attempt to see the prediction overall.

## 2.1 Training a linear regression model to predict the target feature.

In [ ]:
linear_reg = LinearRegression().fit(X_train, y_train)

## 2.2 Printing out the coefficients learned by the model and discussing the role in the model.

In [ ]:
print("\nThe features are: \n", X_train.columns)
print("\nThe coefficients are: \n", linear_reg.coef_)
print("\n The intercept is: \n", linear_reg.intercept_)
print("\nFeatures and coefficients: \n", list(zip(X_train.columns, linear_reg.coef_[0])))

## 2.3 Printing the predicted target feature. Printing the predicted class for a few examples. Printing classification evaluation measures computed on the full training set. 

In [ ]:
# Calculating the prediction and threshold value. 
linear_predictions_train_data = (linear_reg.predict(X_train))

In [ ]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([y_train, pd.DataFrame(linear_predictions_train_data, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg.head(50))

In [ ]:
# Printing the a few classification evaluation measures computed on the full training set.
# The following will be printed: accuracy, confusion matrix, precision, recall, f1).
# Some more evaluation metrics.
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_train, linear_predictions_train_data)))
print("MSE Score: ", metrics.mean_squared_error(y_train, linear_predictions_train_data))
print("MAE Score: ", metrics.mean_absolute_error(y_train, linear_predictions_train_data))
print("R2 Score: ", metrics.r2_score(y_train, linear_predictions_train_data))

## 2. 4 Evaluating the model using classification evaluation measures on the hold-out (30% examples) test set.

In [ ]:
linear_predictions_testing = (linear_reg.predict(X_test))

print("\nPredictions that has multiple linear regression: \n")
actual_vs_predicted_multiple_linear_reg = pd.concat([y_test, pd.DataFrame(linear_predictions_testing, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiple_linear_reg)

In [ ]:
# Printing the a few classification evaluation measures computed on the full training set.
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_test, linear_predictions_testing)))
print("MSE Score: ", metrics.mean_squared_error(y_test, linear_predictions_testing))
print("MAE Score: ", metrics.mean_absolute_error(y_test, linear_predictions_testing))
print("R2 Score: ", metrics.r2_score(y_test, linear_predictions_testing))

In [ ]:
# Taking a tripid from 46a and applying a prediction using Linear Regression model.
route_46a = df[(df['TRIPID'] == '8591174') & (df['DAYOFSERVICE']=='2018-12-23')]
route_46a = route_46a.drop(low_corr, 1)
route_46a = pd.get_dummies(route_46a)
actualtimes_46a = pd.DataFrame(route_46a['ACTUALTIME_TRAVEL'])
actualtimes_46a.reset_index(drop=True, inplace=True)
route_46a = route_46a.drop('ACTUALTIME_TRAVEL', 1)

In [ ]:
prediction_46a = linear_reg.predict(route_46a)
prediction_46a[0] = 0.0

In [ ]:
route_46a

In [ ]:
print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted = pd.concat([actualtimes_46a, pd.DataFrame(prediction_46a, columns=['Prediction'])], axis=1, join='outer')
actual_vs_predicted.head(50)

In [ ]:
# Printing evaluation metrics
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(actualtimes_46a, prediction_46a)))
print("MSE Score: ", metrics.mean_squared_error(actualtimes_46a, prediction_46a))
print("MAE Score: ", metrics.mean_absolute_error(actualtimes_46a, prediction_46a))
print("R2 Score: ", metrics.r2_score(actualtimes_46a, prediction_46a))

***

<br><br>

# 3. Route model and taking the proportion of the prediction to calculate a journey time for the user.

## 3.1 Calculating the proportion of each stop from the overall trip.

In [ ]:
def proportion_stops(predictions):
    # Sum from the first stop until each stop
    sum_each_stop = np.zeros(predictions.shape[0], dtype=float)
    proportion_each_stop = np.zeros(predictions.shape[0], dtype=float)
    overall_prediction = np.sum(predictions)
    
    # Adding sum up until current stop and dividing by overall prediction to get proportion of the trip
    for length in range(predictions.shape[0]):
        sum_each_stop = np.append(sum_each_stop, [predictions[length]])
        sum_overall = np.sum(sum_each_stop) / overall_prediction*100
        proportion_each_stop[length] = sum_overall
        
    return proportion_each_stop

## 3.2 Return the progrnumber based off the stoppointid in a route

Finding the most common progrnumber based off the stoppointid. The reason for using to find the most common progrnumber is because it assumes that most route_id for each line would be always complete with the exception of a few trips in which they take a different route and skips some stops as a result.

In [ ]:
# Code taken from https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/

# array only accepts a panda Series or numpy array
def most_common(array):
    List = array.tolist()
    mode_list = mode(List)
    if mode_list == '1':
        return 0
    
    else:
        return(mode(List))

## 3.3 Calculating the journey time from a start to end destination based on user input

Finding the travel time duration based on a stoppointid then getting the progrnumber

In [ ]:
def journey_time(start,end, prediction):
    # Converting into int because the function returns a string
    start_progrnum = int(most_common(df['PROGRNUMBER'][df['STOPPOINTID']==start]))
    end_progrnum = int(most_common(df['PROGRNUMBER'][df['STOPPOINTID']==end]))
    
#     print(start_progrnum)
#     print(end_progrnum)

    proportion_array = proportion_stops(prediction)
    overall_prediction = np.sum(prediction)
    
    # calculating the time difference from start to end destination 
    start_prediction = (proportion_array[start_progrnum]/100) * overall_prediction
    end_prediction = (proportion_array[end_progrnum]/100) * overall_prediction
    
    journeytime = end_prediction - start_prediction
    
    # print(journeytime)
    
    return journeytime

In [ ]:
user_start = '807'
user_end = '812'

journey_time(user_start, user_end, prediction_46a)

***

<br><br>
# 4.  Random Forest & Decision Trees

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfm = RandomForestRegressor(n_estimators=40, oob_score=True, random_state=1)
rfm.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtc_4 = DecisionTreeRegressor(max_depth=4, random_state=1)
dtc_4.fit(X_train, y_train)

In [ ]:
importance = pd.DataFrame({'feature': X_train.columns, 'importance': rfm.feature_importances_})
importance.sort_values('importance', ascending=False)

In [ ]:
# Printing the evaluated mode using the hold-out training set
rfm_predictions_train = rfm.predict(X_train)
predicted_death_yn_Yes = pd.DataFrame(rfm_predictions_train, columns=['Predicted'])
actual_vs_predicted_rfm_train = pd.concat([y_train, predicted_death_yn_Yes], axis=1)
actual_vs_predicted_rfm_train.head(20)

In [ ]:
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_train, rfm_predictions_train)))
print("MSE Score: ", metrics.mean_squared_error(y_train, rfm_predictions_train))
print("MAE Score: ", metrics.mean_absolute_error(y_train, rfm_predictions_train))
print("R2 Score: ", metrics.r2_score(y_train, rfm_predictions_train))

In [ ]:
# Printing the evaluated mode using the hold-out training set
rfm_predictions_test = rfm.predict(X_test)
predicted_death_yn_test = pd.DataFrame(rfm_predictions_test, columns=['Predicted'])
actual_vs_predicted_rfm_test = pd.concat([y_test, predicted_death_yn_test], axis=1)
actual_vs_predicted_rfm_test.head(20)

In [ ]:
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(y_test, rfm_predictions_test)))
print("MSE Score: ", metrics.mean_squared_error(y_test, rfm_predictions_test))
print("MAE Score: ", metrics.mean_absolute_error(y_test, rfm_predictions_test))
print("R2 Score: ", metrics.r2_score(y_test, rfm_predictions_test))

***

<br><br>
# 5. Stop pair model

## 5.1 First version of paired stop approach
<br><br>
This approach makes a model based on the stopid and its previous stopids

In [ ]:
# Returns a paired list of stops
def paired_stops(df):
    stopid = df['STOPPOINTID'].unique().tolist()
    previous_stopid = []
    for i in stopid:
        prev = df['PREVIOUS_STOPPOINTID'][df['STOPPOINTID']==i]
        # Adds most frequent previous stopid to list
        previous_stopid += [prev.value_counts().idxmax()]
    
    return [stopid, previous_stopid]

In [ ]:
for ids in range(len(paired_stops[0])):
    
    # Making new dataframe
    to_add = df[df['STOPPOINTID']==paired_stops[0][ids]]
    to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==paired_stops[1][ids]])
    stops_df = pd.DataFrame(data=to_add)
    
    # Setting target feature
    y = stops_df['ACTUALTIME_TRAVEL']
    
    # Dropping target feature and low corr features
    stops_df = stops_df.drop(low_corr,1)
    stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
    stops_df = pd.get_dummies(stops_df)
    
    # Fitting model based on stops
    linear_reg = LinearRegression().fit(stops_df, y)
    
    # Save to pickle file 

In [ ]:
pair_stops = paired_stops(df)
to_add = df[df['STOPPOINTID']==pair_stops[0][5]]
to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==pair_stops[1][5]])
stops_df = pd.DataFrame(to_add)

 # Setting target feature
y = stops_df['ACTUALTIME_TRAVEL']
    
# Dropping target feature and low corr features
stops_df = stops_df.drop(low_corr,1)
stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
stops_df = pd.get_dummies(stops_df)

# Fitting/Training model based on stops
linear_reg_model_ = LinearRegression().fit(stops_df, y)

# Saving to pickle File
with open('model_'+pair_stops[0][5]+'.pkl', 'wb') as handle:
    pickle.dump(linear_reg_model_, handle)


In [ ]:
sampledf = stops_df.iloc[[0]]
sample_prediction = linear_reg_sample.predict(sampledf)

In [ ]:
sample_prediction

In [ ]:
with open('model_'+pair_stops[0][5]+'.pkl', 'rb') as handle:
    model = pickle.load(handle)

In [ ]:
model.predict(sampledf)

## 5.2.1 Setting up for 46a stop pair models using first approach

In [ ]:
# Function to get previous stopid and return a paired list
def pair_stopids(current_stopids):
    previous_stopid = []
    for i in current_stopids:
        prev = df['PREVIOUS_STOPPOINTID'][df['STOPPOINTID']==i]
        # Adds most frequent previous stopid to list
        previous_stopid += [prev.value_counts().idxmax()]
    
    return [current_stopids, previous_stopid]

In [ ]:
# Loading the json file
import json
file = open('routes_and_stops.json',)
routes_stops = json.load(file)

In [ ]:
# Get all stops for 46a going outbound ('1')
list_46a_stops = routes_stops['46A']['outbound']

# Pairing stopids and prev stopids from 46a route
pairing_46a_stopids = pair_stopids(list_46a_stops)
predictions = []

In [ ]:
for ids in range(len(pairing_46a_stopids[0])):
    # Making new dataframe
    to_add = df[df['STOPPOINTID']==pairing_46a_stopids[0][ids]]
    to_add = to_add.append(df[df['PREVIOUS_STOPPOINTID']==pairing_46a_stopids[1][ids]])
    stops_df = pd.DataFrame(data=to_add)
    
    # Setting target feature
    y = stops_df['ACTUALTIME_TRAVEL']
    
    # Dropping target feature and low corr features
    stops_df = stops_df.drop(low_corr,1)
    stops_df = stops_df.drop('ACTUALTIME_TRAVEL',1)
    stops_df = pd.get_dummies(stops_df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(stops_df, y)
    
      # Save to pickle file
#     with open('model_'+pairing_46a_stopids[0][ids]+'.pkl', 'wb') as handle:
#         pickle.dump(linear_reg_model, handle)

     # Predicting data
    with open('stop_'+pair_stops[0][ids]+'.pkl', 'rb') as handle:
        model = pickle.load(handle)
    
    k = model.predict(route_46a.iloc[[index]])
    predictions += [k]

In [ ]:
# Printing evaluation metrics
print("RMSE Score: ", np.sqrt(metrics.mean_squared_error(actualtimes_46a, predictions)))
print("MSE Score: ", metrics.mean_squared_error(actualtimes_46a, predictions))
print("MAE Score: ", metrics.mean_absolute_error(actualtimes_46a, predictions))
print("R2 Score: ", metrics.r2_score(actualtimes_46a, predictions))

<br><br>
##### Conclusion:
Linear regression model is not very good. MSE score is off by more than 1000 seconds. And the R2 score is at a negative value. This means the parameters need to be tuned. Keeping dwelltime might be good.

## 5.3 Stop pair based on entire leavetimes

<br><br>
1) Make a rough query that selects rows that contain a certain stopid and its previous stopid based on the direction.

In [ ]:
# initialise query - for OUTBOUND (WHERE DIRECTION == '1')
# query_stopid = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.STOPPOINTID = " + current_stopid + " AND leavetimes.DAYOFSERVICE = weather.dt"
# query_stopid_df = pd.read_sql(query_previoustop, conn)

<br><br>
2) Make a function that will combine lists in a list together as one list

In [1]:
def combine_listsoflist(to_combine):
    combined = []
    for each_list in to_combine:
        combined += each_list
    return combined

<br><br>
4) Make a function that will get rid of the duplicates in the list

In [2]:
def get_unique(stopids_list):
    return list(set(stopids_list))

<br><br>
5) Make a list to store all stopids for DIRECTION == outbound/1.

In [3]:
# Loading the json file
import json
file = open('routes_and_stops.json',)
routes_stops = json.load(file)

In [4]:
# Looping through every lineid, outbound 
stopids_outbound = []
for i,j in routes_stops.items():
    try:
#         print(i, '\n', routes_stops[i]['outbound'], '\n')
        stopids_outbound += [routes_stops[i]['outbound']]
    except KeyError:
        continue
        
# Calling function to get combined list
combined_stopids_outbound = combine_listsoflist(stopids_outbound)

# Calling function to get unique stopids from combined list
unique_stopids_outbound = get_unique(combined_stopids_outbound)

<br><br>
6) Make a list to store all stopids for DIRECTION ==inbound/2.

In [ ]:
# Looping through every lineid, inbound
stopids_inbound = []
for i,j in routes_stops.items():
    try:
#         print(i, '\n', routes_stops[i]['inbound'], '\n')
        stopids_inbound += [routes_stops[i]['inbound']]
    except KeyError:
        continue
        
# Calling function to get combined list
combined_stopids_inbound = combine_listsoflist(stopids_inbound)

# Calling function to get unique stopids from combined list - using set() to get rid off existing stops from outbound stops
unique_stopids_inbound = list(set(combined_stopids_inbound) - set(combined_stopids_outbound))

<br><br>
7) Query to select all of the previous_stopids based on the current stopid and put it to a list

In [ ]:
# query_previoustop = "SELECT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + current_stopid 
# query_prevstop_df = pd.read_sql(query_previoustop, conn)

# # Converting into a pandas series then to list
# query_prevstop_series = query_prevstop_df.iloc[0]
# query_prevstop_list = query_prevstop_series.tolist()

<br><br>
8) Query to select the rows based on the previous stopids and append them to the current dataframe of the current stopid


In [5]:
def df_prev_stops(query_prevstop_list):
    query_prevstop_rows = "SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN " + str(query_prevstop_list)
    print(query_prevstop_rows)
    df_prevstop = pd.read_sql(query_prevstop_rows, conn)
    return df_prevstop

In [6]:
def df_prev_stops_one_element(query_prevstop_list):
    query_prevstop_rows = "SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = " + str(query_prevstop_list)
    print(query_prevstop_rows)
    df_prevstop = pd.read_sql(query_prevstop_rows, conn)
    return df_prevstop

<br><br>
9) Adding index on STOPPOINTID and PREVIOUS_STOPPOINTID

In [ ]:
# Adding indexes
# add_index1 = """CREATE INDEX stopid ON leavetimes(STOPPOINTID);"""
# add_index2 = """CREATE INDEX previous_stopid ON leavetimes(PREVIOUS_STOPPOINTID);"""
# conn.execute(add_index1)
# conn.execute(add_index2)

<br><br>
10) Piecing every step together

In [15]:
# Lists all stops done so far. This is for when laptop needs to rest
import os 
arr = os.listdir('C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/stop_models')
j = []
for i in arr:
    j += i.split('_')

h = []
for i in j:
    h += i.split('.')

g = [str(i) for i in h if i.isdigit()]
unique_stopids_outbound = [x for x in unique_stopids_outbound if x not in g]
len(unique_stopids_outbound)

15

In [ ]:
# previous_stops = {}
# for stopid in unique_stopids_outbound:
#     # Get all previous stopids in list
#     query_previoustop = "SELECT DISTINCT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + stopid
#     query_prevstop_df = pd.read_sql(query_previoustop, conn)

#     # Converting into a pandas series then to list
#     query_prevstop_series = query_prevstop_df['PREVIOUS_STOPPOINTID'].tolist()
#     query_prevstop_list = [stopid for stopid in query_prevstop_series if stopid != '0']
#     previous_stops[stopid] = query_prevstop_list
#     print('finished')
    
# with open('previous_stops_outbound.json', 'w+') as fp:
#     json.dump(previous_stops, fp)

In [8]:
# import boto3
import pandas as pd
import numpy as np
import sqlite3
import pickle

# from sagemaker import get_execution_role
from sklearn.linear_model import LinearRegression
from math import log
from multiprocessing import Pool


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Connecting to s3
# role = get_execution_role()
# bucket='sagemaker-studio-520298385440-7in8n1t299'
# data_key = 'route_46a.feather'
# data_location = 's3://{}/{}'.format(bucket, data_key)

In [9]:
low_corr = ['DAYOFSERVICE', 'VEHICLEID', 'TRIPID', 'STOPPOINTID', 'PREVIOUS_STOPPOINTID', 'PROGRNUMBER', 'temp', 'pressure', 'humidity', 
            'wind_speed', 'wind_deg', 'weather_id', 'weather_description', 'clouds_all', 'PREVIOUS_STOPPOINTID', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP',
           'PLANNEDTIME_TRAVEL', 'DWELLTIME']

In [10]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except 'Error' as e:
        print(e)
        
    return conn

In [11]:
# create connection to db
db_file = "C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/dublinbus.db"
conn = create_connection(db_file)

In [14]:
# Outbound
file = open('previous_stops.json',)
previous_stops = json.load(file)

# Query to get all of weather
weather_query = "SELECT weather.* from weather"
weather_df = pd.read_sql(weather_query, conn)
weather_df = weather_df.rename(columns={"dt": "DAYOFSERVICE"})

for current_stopid in unique_stopids_outbound:
    
    query_prevstop_series = previous_stops[current_stopid]
    query_prevstop_list = tuple(query_prevstop_series)
    if len(query_prevstop_list) == 1:
        # Making query to db and make df
        query_stopid = "SELECT leavetimes.* FROM leavetimes WHERE leavetimes.STOPPOINTID = " + current_stopid + "OR leavetimes.PREVIOUS_STOPPOINTID = " + str(query_prevstop_list)
        df = pd.read_sql(query_stopid, conn)
        
        # Append previous stops rows to main df
#         to_add = df_prev_stops_one_element(query_prevstop_series[0])
#         df = pd.concat([df,to_add])
        df = df.merge(weather_df, on='DAYOFSERVICE', how='left')
        
    elif len(query_prevstop_list) == 0:
        continue
        
    else:
        # Making query to db and make df
        query_stopid = "SELECT leavetimes.* FROM leavetimes WHERE leavetimes.STOPPOINTID = " + current_stopid + "OR WHERE leavetimes.PREVIOUS_STOPPOINTID IN " + str(query_prevstop_list)"
        df = pd.read_sql(query_stopid, conn)
        
        # Append previous stops rows to main df
#         to_add = df_prev_stops(query_prevstop_list)
#         df = pd.concat([df,to_add]) 
        df = df.merge(weather_df, on='DAYOFSERVICE', how='left')

    # Drop low correlated features and setting target feature
    df = df.drop(low_corr, 1)
    tf = df['ACTUALTIME_TRAVEL']
    df = df.drop('ACTUALTIME_TRAVEL', 1)
    df = pd.get_dummies(df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(df, tf)
    
    # Save to pickle file
    with open('C:/Users/fayea/UCD/ResearchPracticum/Data-Analytics-CityRoute/stop_models/stop_'+ current_stopid +'.pkl', 'wb') as handle:
        pickle.dump(linear_reg_model, handle)
    
    print('Finished.')

SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2039', '7472', '3230')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2950', '7447', '303', '2981', '2825', '1315', '2984', '7449', '2897', '2955', '4988', '7067', '2965', '2983', '804', '4983', '2934', '399', '2954', '4982', '2933', '4332')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('132', '166', '4891', '2492', '165', '131', '6228', '6273', '332', '128', '7131', '324', '195', '2975', '91', '149')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = 684
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1701', '6094', '4715', '7339', '2', '820', '4523', '1814', '6282', '1696', '1700', '6317', '4525', '4322', '315', '3802', '4', '3135', '7340', '4387', '767', '7574', '288', '449', '6348', '877', '2270', '300', '6071')
Finished.
SELECT leavetimes.* FROM 

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('6053', '1184', '6051', '3605', '3606', '6052')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2611', '2420')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1509', '7027', '7', '5144', '1506', '199', '7072', '1507', '1508', '14', '274', '198', '4316')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = 1133
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2239', '2211', '7374', '332', '3890', '7390', '7049', '4616', '2210', '4003', '4', '7387', '5114', '3374', '6348', '4953', '763', '2166', '1479', '315', '1358', '6089', '2232', '406', '4962', '4320', '6317', '1104', '4620', '7571', '7459', '3917', '5192', '1842', '6289', '7162', '7025', '110', '3874', '5056', '3085', '4715', '7012')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetime

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('6278', '354', '6277', '399', '6280', '7581', '395', '1077')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('3280', '2045', '7360', '4313', '3278', '3471', '7372', '4773', '2039', '3283', '2810')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('776', '773', '6122', '2039', '775', '767', '4320', '6089', '3146', '774', '760', '769', '772', '2065', '768', '2060', '7574', '807', '763', '7390', '7387', '7587', '7160', '2041', '7339', '264', '7162')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('4249', '4248')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1310', '2825', '2954', '2955', '2965', '2967', '1308', '1309')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('3398', '3384', '4495', '3397', '7483',

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2165', '4668', '4669', '4677', '6244', '327', '7229', '4670', '3379', '2137')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1633', '324', '21', '7479', '7485', '7484', '5028', '3699', '7486', '288', '3885', '7487', '4957', '7347', '292', '2975', '3669', '303', '332', '7349', '217', '4843')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('335', '336', '271', '1490', '4495', '4322', '511', '4392', '1375', '7390', '6056', '2039', '2103', '274', '7131', '263', '277', '6184', '6094', '2102', '4616', '272', '7547', '7546', '191', '4595', '2063', '2166', '7226', '7665', '52', '513', '4513', '84', '51', '2418', '288', '5192', '4525', '7174', '1491', '187', '6317', '190', '3399', '6071', '819', '1358', '3362', '749', '2002', '7011')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2801'

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('225', '15', '278', '7602', '205', '85', '381', '224', '220', '12', '400', '7615', '19', '14', '18', '319', '223', '1651', '1620', '17', '21', '222', '399', '395', '10', '392', '371', '7571', '396')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('4626', '4620', '3368', '7563', '7459')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2678', '4795', '6348')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2789', '2788')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2783', '2650', '2328', '2908', '326', '6029', '2648', '850')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2964', '7522', '2962', '4131', '4171', '7574', '2963', '4173', '4320', '4172')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE

...

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2279', '7514', '4619', '2278')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('7343', '7124', '7243', '1580')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('4894', '1675', '7588', '2959')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2217', '6089', '1445', '4401', '2214')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1181', '332', '2492', '1555', '324', '6071', '1554', '1557', '1556')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1679', '2959')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2129', '2125', '2126', '4795', '4341', '2624', '2675', '2243', '2124')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('4

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('629', '628', '627', '626')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1999', '1998', '6056', '1466', '6057', '1490', '4795', '4410', '7229', '4576', '4513', '1989', '2719', '6348', '1992', '1982', '4400', '6234', '4715')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1947', '1988', '1935', '1982', '261', '4715', '1938', '49', '335')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2518', '281', '1934', '7459', '342', '1078', '2517')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('3209', '3208', '4962')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('42', '41', '449', '40', '6122', '4848')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = 2830
Finished.
SELE

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('2009', '2052', '7564', '760', '2008', '807', '4320', '7574', '5016', '334', '757', '2039', '2007', '761', '2083', '848', '747', '762', '406', '866', '763', '765', '1476', '3085', '3134', '767', '6000', '846', '759')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('7149', '895', '3874', '1508', '3333', '3544', '7348', '313', '1285', '335', '319', '927', '6059', '264', '4387', '7226', '7181', '6089', '2657', '7', '2039', '5192', '4411', '288', '7231', '315', '1749', '2', '4616', '4982', '4957', '2622', '3802', '1022', '6317', '4392', '2575')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('6171', '6039', '2294', '1816', '4747', '1012', '7025')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('66', '5147', '820', '60', '63', '59', '3')
Finished.
SELECT leavetimes.* FROM leavetimes WHE

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('5106', '7360', '7324', '2039', '3283', '3309', '7372')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1890', '7025', '1899', '1889')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('759', '7574', '758', '807', '2039', '4320', '7622', '406', '819', '4725', '810', '334', '818', '1476', '1479', '776', '747', '264', '6059', '336', '4982', '2795', '6110', '927', '2', '2199', '4132', '315', '3135', '812', '845', '808', '4619', '4956', '760', '320', '814', '5090', '804', '848', '299', '763', '7453', '303', '786', '278', '806', '81', '4151', '811', '4307', '7353', '435', '846', '7588', '335', '907', '771', '354', '585', '772', '774', '2007', '7513', '5092', '813', '767', '773', '809', '7638', '6137', '2009', '847', '842', '757')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1439', '4

Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('5140', '675', '6251', '2279', '2355', '7514', '302', '4595', '4596', '4619', '666', '404', '618')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('3641', '4381', '951', '7382', '2212', '1073', '3616', '3590', '949', '1184', '497')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1008', '1007')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = 1601
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('1997', '1995', '1939', '6056', '6348', '4515', '274', '1490', '19', '6057', '1468', '1996', '6234', '2689', '7229', '1378', '2713', '2205', '332', '4795')
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID = 3208
Finished.
SELECT leavetimes.* FROM leavetimes WHERE leavetimes.PREVIOUS_STOPPOINTID IN ('4571', '4320', '435',

In [ ]:
# Inbound 
for current_stopid in range(len(stopids_inbound)):
    # Making query to df and make df
    query_stopid = "SELECT leavetimes.*, weather.* FROM leavetimes, weather WHERE leavetimes.STOPPOINTID = " + stopids_inbound[current_stopid] + " AND leavetimes.DAYOFSERVICE = weather.dt"
    df = pd.read_sql(query_previoustop, conn)
    
    # Get all previous stopids in list
    query_previoustop = "SELECT leavetimes.PREVIOUS_STOPPOINTID FROM leavetimes WHERE leavetimes.STOPPOINTID = " + stopids_inbound[current_stopid] 
    query_prevstop_df = pd.read_sql(query_previoustop, conn)

    # Converting into a pandas series then to list
    query_prevstop_series = query_prevstop_df.iloc[0]
    query_prevstop_list = query_prevstop_series.tolist()
    
    # Append previous stops rows to main df
    to_add = df_prev_stops(query_prevstop_list)
    df = df.append(to_add)
    
    # Drop low correlated features and setting target feature
    df = df.drop(low_corr, 1)
    tf = df['ACTUALTIME_TRAVEL']
    df = df.drop('ACTUALTIME_TRAVEL', 1)
    df = pd.get_dummies(df)
    
    # Fitting model based on stops
    linear_reg_model = LinearRegression().fit(df, tf)
    
    # Save to pickle file
    with open('/UCD/ResearchPracticum/Data-Analytics-CityRoute/stop_models/stop_'+ stopids_outbound[current_stopid] +'.pkl', 'wb') as handle:
        pickle.dump(linear_reg_model, handle)

***